## 标签

from curses import keyname


key 这里有 reasoning chat_stage dialog 三个keyname 相比于纯中文多的还有“标题"
content 这里是对话或reasoning 的具体内容
source 这里是对话的来源,就是书籍的来源吧，这个随便挑选就可以
mapping 这里是对话的映射关系 有总结上文，承上启下，引出下文等关系 这个标签只有 reasoning才有，这个需要用gpt去生成
method 具体是什么方法：这个标签只有
dialog 这是一个列表，列表中对话单轮次的标识，有speaker, origin_text 等标识

Participant 等标识


## 先这样处理

In [305]:
import regex
import os
import re
input_path="/home/ckqsudo/code2024/0dataset/E-bed/English/z1_datastream_划分reasoning和对话/输入文件夹"
files=os.listdir(input_path) # 读取文件夹目录
res=[]
files

['Cognitive Behaviour Therapy 100_signed.txt',
 'Doing Something Different Solution-Focused Brief Therapy Practices_signed.txt',
 'Acceptance  Commitment Therapy (ACT) 100_signed.txt',
 'Solution-Focused Brief Therapy with Families_signed.txt',
 'Solution-Focused Brief Therapy in Schools_signed.txt',
 'Solution focused brief therapy_100_signed.txt',
 'Brief Counseling That Works A Solution-Focused Therapy Approach for School Counselors_signed.txt',
 'Doing Psychotherapy_signed.txt',
 'Becoming Solution-Focused In Brief Therapy_signed.txt',
 'Gestalt Therapy 100_signed.txt',
 'More than miracles_ the state of the art of solution-focused brief therapy_signed.txt',
 'Rational Emotive Behavior Therapy 100_signed.txt']

In [316]:
paricipant=['George', 'Mrs. D',  'Emma', 'Dan', 'Phil', 'Hamid', 'Paul',  'Mother', 'Fam', 'Daughter', 'Fancy', 'Eleanor', 'Bob', 'Korley', 'Shantel', 'Beth', 'Worker', 'Wife', 'Sara', 'Mn Arianes',  'Josie', 'Susan', 'Bry', 'Boy', 'Robert', 'Mr. Arianes', 'Mr. Arians',  'Parents', 'Jake', 'Sherri', 'Mac',   'Student 2', 'Young person', 'Father', 'Ms. Valdez', 'James',  'Cora', 'Jessica',  'Student', 'Tom',  'Shawn', 'Counselor', 'Jasmine', 'Karen', 'Sandra', 'Miss Banks', 'Student 4', 'Anita', 'Scott', 'Jackie', 'Janet', 'Margaret', 'Marie', 'Student 3', 'Example', 'Carl', 'Mr. D', 'M’Lin', 'Both', 'Dick', 'Emily', 'Janine', 'Dorothy', 'Pete', 'Ms. Wilson',  'Resident', 'Sarah', 'Students', 'Brad', 'Client', 'All', 'Pedro', 'Ben and Paul', 'Student 1', 'Mum', 'Kasey', 'Mr. Smith', 'Victoria', 'Jamel',  'Christine', 'Ben','Mrs. Smith (MS)','Mrs. Smith','Mrs. Smith','Mr. Daniels (Mr. D)','Mrs. Daniels (Mrs. D)','Peter','Sophie','Mrs. Daniels (Mrs. D)','Mrs. Daniels']

In [317]:
# Acceptance  Commitment Therapy (ACT) 100

source_method_mapping={'Cognitive Behaviour Therapy 100_signed.txt':"Cognitive Behaviour Therapy",
 'Doing Something Different Solution-Focused Brief Therapy Practices_signed.txt':"Solution-Focused Brief Therapy",
 'Acceptance  Commitment Therapy (ACT) 100_signed.txt':"Acceptance  Commitment Therapy",
 'Solution-Focused Brief Therapy with Families_signed.txt':"Solution-Focused Brief Therapy",
 'Solution-Focused Brief Therapy in Schools_signed.txt':"Solution-Focused Brief Therapy",
 'Solution focused brief therapy_100_signed.txt':"Solution-Focused Brief Therapy",
 'Brief Counseling That Works A Solution-Focused Therapy Approach for School Counselors_signed.txt':"Solution-Focused Brief Therapy",
 'Doing Psychotherapy_signed.txt':"Unknown",
 'Becoming Solution-Focused In Brief Therapy_signed.txt':"Solution-Focused Brief Therapy",
 'Gestalt Therapy 100_signed.txt':"Gestalt Therapy",
 'More than miracles_ the state of the art of solution-focused brief therapy_signed.txt':"Solution-Focused Brief Therapy",
 'Rational Emotive Behavior Therapy 100_signed.txt':"Rational Emotive Behavior Therapy"}

In [308]:
printed_methods=['Gestalt Therapy', 'Rational Emotive Behavior Therapy', 'Unknown', 'Acceptance  Commitment Therapy', 'Solution-Focused Brief Therapy', 'Cognitive Behaviour Therapy']

In [309]:
noisy=['8\n',
 '(Task)\n',
 'Sessions 26-55\n',
 '41\n',
 'Scaling questions.\n',
 'Laura,\n',
 'Example 1\n',
 '\t\n',
 '(Summary statement)\n',
 'Session 1\n',
 'Session 6\n',
 '40\n',
 'Solution-focused goals.\n',
 'Example 3\n',
 'Chapter 1\n',
 'or.\n',
 '(1993: 29)\n',
 '92\n',
 'Session 22\n',
 'Session 85\n',
 '15\n',
 'Chapter 7\n',
 '\n',
 '(Scaling)\n',
 'Sessions 17-70\n',
 '(Message)\n',
 '12\n',
 'Exceptions elicitation:\n',
 '20\n',
 'Becky\n',
 'Sessions 6-31\n',
 'Final Thoughts\n',
 '26\n',
 'Session 32\n',
 'EARS\n',
 '(Cheerleading)\n',
 '(Mindmapping)\n',
 '141\n',
 ' \n',
 '27\n',
 'Session 5\n',
 '10\n',
 'Session Five\n',
 '200\n',
 'Session 76\n',
 '75\n',
 '123',
 'CASE EXAMPLE\n',
 'Coping questions.\n',
 'Miracle question.\n',
 'Session 2\n',
 'Session 73\n',
 'Frank Thomas\n',
 'It’s weird.\n',
 '95\n',
 '[Pause.]\n',
 'Sessions 77-84\n',
 'or:\n',
 'Session 56\n',
 '89\n',
 'Positive Goals\n',
 'SUBSEQUENT SESSIONS\n',
 'Session 4\n',
 '16\n',
 '9\n',
 'Scaling\n',
 '14\n',
 'Session 34\n',
 'Chapter 5\n',
 'Compliments\n',
 '77\n',
 'Chapter 4\n',
 '29\n',
 'Session 3\n',
 'Specification:\n',
 'SPACING SESSIONS\n',
 'Eliciting:\n',
 'Pre-session change.\n',
 '(Detailing)\n',
 'Intervention Prompts\n',
 'Example\n',
 'Compliments.\n',
 '65\n',
 'Session Four\n',
 '“What else?”\n',
 '17\n',
 'Session Two\n',
 'Reinforcing:\n',
 'Session 25\n',
 '(1991a: 181)\n',
 'EXERCISE\n',
 'Session One\n',
 'Chapter 2\n',
 '35\n',
 'Chapter 9\n',
 '(Miracle question)\n',
 'Session 24\n',
 '31\n',
 '(Compliments)\n',
 '30\n',
 'Experiential exercise\n',
 '62\n',
 '* * *\n',
 'Session 66\n',
 'Session 33\n',
 'Sessions 67-70\n',
 'Example 2\n',
 '4\n',
 'Summary\n',
 'Asking directly.\n',
 'Session Three\n',
 'Negative Goals\n',
 '13\n',
 'Chapter 3\n',
 'Specific Interventions\n',
 'Session 57\n',
 'Case Examples\n',
 'Session 23\n',
 'Sessions 4-21\n',
 'Bridging Statement\n',
 '11\n',
 'Amplifying:\n']

In [310]:
methods=[]
for i in files:
    methods.append(source_method_mapping[i])
print(list(set(methods)))


['Gestalt Therapy', 'Rational Emotive Behavior Therapy', 'Unknown', 'Acceptance  Commitment Therapy', 'Solution-Focused Brief Therapy', 'Cognitive Behaviour Therapy']


In [311]:
chat_supporter_sig=regex.compile(r"Therapist: .*\n")
chat_participant_sig=regex.compile(r"Client: .*\n|^.{0,15}:.*\n")
title_sig=regex.compile(r"^###.*\n")
stage_sig=regex.compile(r"第一次会谈.*\n|周期会谈.*\n|早期会谈.*\n|第一次会面.*\n|周期会面.*\n|早期会面.*\n")


In [312]:
from email.message import EmailMessage

def idata_main_process(idata:list,source_file:str,final_res:list):
    dialog_list={'key':'fewshot','content':'','source':source_file,'method':source_method_mapping[source_file],'dialog':[]}
    for idx,line in enumerate(idata):
        if line=="\n" or line=="\n\t" or len(line)<10 or len(line.split(" "))<2 or line in noisy:
            continue
        if chat_supporter_sig.findall(line): #咨询师
            line=line[:-1] if line[-1]=="\n" else line
            dialog_list['dialog'].append({"speaker":"supporter","origin_text":line})
        elif chat_participant_sig.findall(line) and line.split(":")[0] in paricipant:# 其他人
            line=line[:-1] if line[-1]=="\n" else line
            dialog_list['dialog'].append({"speaker":"seeker","origin_text":line})
        else:
            if len(dialog_list['dialog'])>0:# 添加日常对话
                final_res.append(dialog_list)
            dialog_list={'key':'fewshot','content':'','source':source_file,'method':source_method_mapping[source_file],'dialog':[]}

            entity={'key':'fewshot','content':'','source':source_file,'method':source_method_mapping[source_file]}
            if title_sig.findall(line):# 作为文章的标题
                entity['key']='title'
                entity['content']=line[3:]
            elif stage_sig.findall(line):# 找到宇华标注的序号
                entity['key']='chat_stage'
                entity['content']=line
            elif len(line)>10 and len(line.split(" ")) >2:# 作为 reasoning
                entity['key']='reasoning'
                entity['content']=line
            else:
                continue # 直接忽略的噪音
            final_res.append(entity)
    if dialog_list['dialog']:# 添加日常对话
        final_res.append(dialog_list)
# stage_sig=regex.compile(r"第一次会面.*\n|周期会面.*\n")
# 非对话数据

In [299]:
def print_participant(idata:list,parti_list:list):
    for idx,line in enumerate(idata):
        if chat_participant_sig.findall(line):
            if chat_supporter_sig.findall(line): #咨询师
                pass
            elif len(line.split(":")[0])>3 in paricipant:# 其他人
                parti_list.append(line.split(":")[0])
    return parti_list


In [300]:
noisy=[]
mis_reason=[]
def test_main_process(idata:str,source_file:str,final_res:list):
    dialog_list={'key':'fewshot','content':'','source':source_file,'method':source_method_mapping[source_file],'dialog':[]}
    for idx,line in enumerate(idata):
        if chat_supporter_sig.findall(line): #咨询师
            # print("咨询师",line)
            pass
        elif chat_participant_sig.findall(line) and line.split(":")[0] in paricipant:# 其他人
            print("来访者",line)
            
        else:
            if title_sig.findall(line):# 作为文章的标题
                pass
                # print('title',line)
            elif stage_sig.findall(line):# 找到宇华标注的序号
                # print('chat_stage',line)
                pass
            elif len(line)>10 and len(line.split(" ")) >2:# 作为 reasoning
                # print('reasoning',line)
                if idx>0 and idx<len(idata) and chat_participant_sig.findall(idata[idx-1]):
                    # print('reasoning可能有问题',line)
                    mis_reason.append(line)
            else:
                # print("噪音",line)
                if idx>0 and idx+1<len(idata) and chat_participant_sig.findall(idata[idx+1]):
                    pass # 不可忽略的噪音
                noisy.append(line)
                continue # 直接忽略的噪音


In [313]:
# parti_list=[]
final_result=[]
for f in files[:]:
    file_position=input_path+"/"+f
    ifile=open(file_position,"r",encoding='utf-8')
    idata=ifile.readlines()
    print(f,len(idata))
    idata_main_process(idata,f,final_result)
    # parti_list=print_participant(idata,parti_list)# 打印所有participant的称呼
    # test_main_process(idata,f,final_result)
# print("噪音",list(set(noisy)))
# print("错误 reasoning",mis_reason)

Cognitive Behaviour Therapy 100_signed.txt 701
Doing Something Different Solution-Focused Brief Therapy Practices_signed.txt 199
Acceptance  Commitment Therapy (ACT) 100_signed.txt 125
Solution-Focused Brief Therapy with Families_signed.txt 701
Solution-Focused Brief Therapy in Schools_signed.txt 772
Solution focused brief therapy_100_signed.txt 549
Brief Counseling That Works A Solution-Focused Therapy Approach for School Counselors_signed.txt 2360
Doing Psychotherapy_signed.txt 2875
Becoming Solution-Focused In Brief Therapy_signed.txt 2305
Gestalt Therapy 100_signed.txt 55
More than miracles_ the state of the art of solution-focused brief therapy_signed.txt 4699
Rational Emotive Behavior Therapy 100_signed.txt 69


In [323]:
len(final_result)

2824

In [324]:
count=0
for i in final_result:
    if i['key']=='fewshot':
        count+=len(i['dialog'])

In [325]:
count

6009

In [315]:
import json
# test=[{"a":"b"},{"c":"d"}]
with open("/home/ckqsudo/code2024/0dataset/E-bed/English/z1_datastream_划分reasoning和对话/z2划分后的json文件/English_Dataset.json","w",encoding="utf-8") as o_f:
    # final_json=json.dumps(final_result)
    # o_f.write(final_json,ensure_ascii=False)
    json.dump(final_result,o_f,ensure_ascii=False)

In [201]:
print(list(set(parti_list)))

['George', 'Mrs. D', 'Emma', 'Dan', 'Phil', 'Hamid', 'Paul', 'Mother', 'Fam', 'Daughter', 'Fancy', 'Eleanor', 'Bob', 'Korley', 'Shantel', 'Beth', 'Worker', 'Wife', 'Sara', 'Mn Arianes', 'Josie', 'Susan', 'Bry', 'Boy', 'Robert', 'Mr. Arianes', 'Mr. Arians', 'Parents', 'Jake', 'Sherri', 'Mac', 'Student 2', 'Young person', 'Father', 'Ms. Valdez', 'James', 'Cora', 'Jessica', 'Student', 'Tom', 'Shawn', 'Jasmine', 'Karen', 'Sandra', 'Miss Banks', 'Student 4', 'Anita', 'Scott', 'Jackie', 'Janet', 'Margaret', 'Marie', 'Student 3', 'Example', 'Carl', 'Mr. D', 'M’Lin', 'Both', 'Dick', 'Emily', 'Janine', 'Dorothy', 'Pete', 'Ms. Wilson', 'Resident', 'Sarah', 'Students', 'Brad', 'Client', 'All', 'Pedro', 'Ben and Paul', 'Student 1', 'Kasey', 'Mr. Smith', 'Victoria', 'Jamel', 'Christine', 'Ben']


In [200]:
list(set(noisy))

['8\n',
 '(Task)\n',
 'Sessions 26-55\n',
 '41\n',
 'Scaling questions.\n',
 'Laura,\n',
 'Example 1\n',
 '\t\n',
 '(Summary statement)\n',
 'Session 1\n',
 'Session 6\n',
 '40\n',
 'Solution-focused goals.\n',
 'Example 3\n',
 'Chapter 1\n',
 'or.\n',
 '(1993: 29)\n',
 '92\n',
 'Session 22\n',
 'Session 85\n',
 '15\n',
 'Chapter 7\n',
 '\n',
 '(Scaling)\n',
 'Sessions 17-70\n',
 '(Message)\n',
 '12\n',
 'Exceptions elicitation:\n',
 '20\n',
 'Becky\n',
 'Sessions 6-31\n',
 'Final Thoughts\n',
 '26\n',
 'Session 32\n',
 'EARS\n',
 '(Cheerleading)\n',
 '(Mindmapping)\n',
 '141\n',
 ' \n',
 '27\n',
 'Session 5\n',
 '10\n',
 'Session Five\n',
 '200\n',
 'Session 76\n',
 '75\n',
 '123',
 'CASE EXAMPLE\n',
 'Coping questions.\n',
 'Miracle question.\n',
 'Session 2\n',
 'Session 73\n',
 'Frank Thomas\n',
 'It’s weird.\n',
 '95\n',
 '[Pause.]\n',
 'Sessions 77-84\n',
 'or:\n',
 'Session 56\n',
 '89\n',
 'Positive Goals\n',
 'SUBSEQUENT SESSIONS\n',
 'Session 4\n',
 '16\n',
 '9\n',
 'Scaling\